TRADE IN GOODS STATISTICS:Employee size_value

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
%run "fetch_source.ipynb"

In [3]:
tab = loadxlstabs(inputFile)

Loading in/idbr_ots_tables_2015.xls which has size 174080 bytes
Table names: ['Notes and Contents', 'Industry Group', 'Age Group', 'Employee Size', 'Industry_Age', 'Industry_EmployeeSize', 'EmployeeSize_Age', 'Metadata']


In [4]:
tab = tab[5]

In [5]:
savepreviewhtml(tab)

TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,,,,,,
,,,,,,,,,,,,,
Release Date 24/11/2016,,,,,,,,,,,,,
,,,,,,,,,,,,,
"Total value of UK Trade, Business count and Employee count, by Industry group and Employee Size group for Imports and Exports",,,,,,,,,,,,,
,,,Exports,,,,Imports,,,,,,
,,,2015.0,,,,2015.0,,,,,,
Industry Group,Employees,,Value £ m,Business Count,Employee Count,,Value £ m,Business Count,Employee Count,,,,
,,,,,,,,,,,,,
Group 1 Agriculture and Food,0.0,,121.0,647.0,0.0,,588.0,1841.0,0.0,,,,
,1 to 9,,439.0,2725.0,10464.0,,594.0,4846.0,17472.0,,,,


In [6]:
observations = tab.excel_ref('D10').expand(DOWN).expand(RIGHT).is_not_blank() 

In [7]:
observations = observations - tab.excel_ref('E10').expand(DOWN).is_not_blank()

In [8]:
observations = observations - tab.excel_ref('I10').expand(DOWN).is_not_blank()

In [9]:
observations = observations - tab.excel_ref('F10').expand(DOWN).is_not_blank()

In [10]:
observations = observations - tab.excel_ref('J10').expand(DOWN).is_not_blank()

In [11]:
Industrygroup = tab.excel_ref('A10').expand(DOWN).is_not_blank()

In [12]:
employees = tab.excel_ref('B10').expand(DOWN)

In [13]:
Flow = tab.excel_ref('D6').expand(RIGHT).is_not_blank()

In [14]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDimConst('Year','2015'),
            HDimConst('Unit', '£ Million'), 
            HDimConst('Measure Type','Total Turnover'),            
            HDim(Industrygroup, 'Industry', CLOSEST, ABOVE),
            HDim(Flow, 'Flow', CLOSEST, LEFT),
            HDim(employees, 'Employment', DIRECTLY, LEFT)
]

In [15]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [16]:
savepreviewhtml(c1)

In [17]:
new_table = c1.topandas()

In [18]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)

In [19]:
new_table['Services'] = new_table['Industry'].map(lambda cell:cell.replace('Grand Total9', 'Grand Total'))

In [20]:
new_table['Services'] = new_table['Services'].str.lstrip('Group 1234567890')

In [21]:
new_table['Services'] = new_table['Services'].map(lambda cell: cell.replace('and', ''))

In [22]:
new_table = new_table[new_table['Value'] != '']

In [23]:
new_table = new_table[new_table['Value'] != 'S']

In [24]:
new_table['Flow'] = new_table['Flow'].str.rstrip('s')
new_table['Value'] = new_table['Value'].astype(int)

In [25]:
new_table['Employment'] = new_table['Employment'].map(str) + ' employees'

In [26]:
new_table['Employment'] = new_table['Employment'].map(lambda cell: cell.replace('0.0', '0'))

In [27]:
# new_table['Services'] = new_table['Industry'].map(lambda x: str(x)[8:])

In [28]:
# new_table['Product'] = new_table['Industry'].map(lambda x: str(x)[0:8])

In [29]:
new_table['Services'] = new_table['Services'].str.strip()
# new_table['Product'] = new_table['Product'].str.strip()
new_table['Year'] = new_table['Year'].str.strip()
new_table['Flow'] = new_table['Flow'].str.strip()
new_table['Unit'] = new_table['Unit'].str.strip()
new_table['Measure Type'] = new_table['Measure Type'].str.strip()
new_table['Geography'] = new_table['Geography'].str.strip()
new_table['Employment'] = new_table['Employment'].str.strip()

In [30]:
new_table = new_table[['Geography','Year','Services','Flow','Employment','Measure Type','Value','Unit']]

In [31]:
# def user_perc(x):
    
#     if x.strip(' ') == '':
#         return 'NA'
#     else:
#         return x
    
# new_table['Services'] = new_table.apply(lambda row: user_perc(row['Services']), axis = 1)
# new_table['Employment'] = new_table.apply(lambda row: user_perc(row['Employment']), axis = 1)

In [32]:
new_table.head(4)

,Geography,Year,Services,Flow,Employment,Measure Type,Value,Unit
0,K02000001,2015,Agriculture Food,Export,0 employees,Total Turnover,121,£ Million
1,K02000001,2015,Agriculture Food,Import,0 employees,Total Turnover,588,£ Million
2,K02000001,2015,Agriculture Food,Export,1 to 9 employees,Total Turnover,439,£ Million
3,K02000001,2015,Agriculture Food,Import,1 to 9 employees,Total Turnover,594,£ Million


In [33]:
new_table.tail(4)

,Geography,Year,Services,Flow,Employment,Measure Type,Value,Unit
120,K02000001,2015,Unknown,Export,Unknown employees,Total Turnover,11160,£ Million
121,K02000001,2015,Unknown,Import,Unknown employees,Total Turnover,6459,£ Million
122,K02000001,2015,Total,Export,employees,Total Turnover,304768,£ Million
123,K02000001,2015,Total,Import,employees,Total Turnover,411033,£ Million
